In [81]:
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from os import walk
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import image
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [108]:
train_list_df = pd.read_csv('train_val_list.txt', header=None, names=['Image Index'])
test_list_df = pd.read_csv('test_list.txt', header=None, names=['Image Index'])
xray_train_df = pd.read_csv('BBox_List_2017.csv')
xray_test_df = pd.read_csv('Data_entry_2017_v2020.csv')

In [109]:
train_list_df.sample(10)

,Image Index
78422,00026199_000.png
41186,00012681_007.png
40178,00012406_000.png
32515,00010261_004.png
13942,00004458_005.png
18215,00005794_003.png
72013,00022760_000.png
85008,00029572_002.png
63704,00019787_002.png
13661,00004339_002.png


In [110]:
test_list_df.sample(10)

,Image Index
8152,00011925_074.png
3293,00005066_024.png
20350,00025707_010.png
14318,00017138_076.png
12679,00015605_021.png
16063,00018921_041.png
20223,00025368_031.png
4918,00007943_002.png
1755,00002437_035.png
13090,00015956_023.png


In [111]:
xray_test_df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,0.168


In [112]:
xray_test_df.columns

Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]'],
      dtype='object')

In [115]:
my_list = []
for each in xray_test_df["Finding Labels"].str.split('|'):
    my_list += each
my_unique_list = set(my_list)

In [114]:
# new = xray_test_df["Finding Labels"].str.split("|", n = 9, expand = True)
# new.sample(20)

In [62]:
# xray_test_df['Finding Labels'] = xray_test_df["Finding Labels"].str.split("|")

In [143]:
def search_me(df_series, term):
    return 1 if term in df_series.split('|') else 0

In [148]:
for each_disease in my_unique_list:
    xray_test_df[each_disease] = xray_test_df["Finding Labels"].apply(search_me, args=(each_disease,))
xray_test_df = xray_test_df.drop('Finding Labels', axis=1)

In [153]:
def my_hot_encoding(dataframe, feature_list_to_encode):
    encoding_dict = {}
    for each_feature in feature_list_to_encode:
        feature_dict = {}
        for index_type, each_type in enumerate(dataframe[each_feature].unique()):
            feature_dict[each_type] = index_type
        encoding_dict[each_feature] = feature_dict
    for each_feature in feature_list_to_encode:
        dataframe[each_feature] = dataframe[each_feature].map(lambda x: encoding_dict[each_feature][x])
    dataframe[each_feature].astype(np.int64)
    return encoding_dict  

In [154]:
gender = my_hot_encoding(xray_test_df, ['Patient Gender'])

In [156]:
view_position = my_hot_encoding(xray_test_df, ['View Position'])

In [157]:
xray_test_df.sample(20)

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Consolidation,Pleural_Thickening,Infiltration,Cardiomegaly,Pneumonia,Hernia,Edema,Fibrosis,Atelectasis,No Finding,Nodule,Effusion,Emphysema,Pneumothorax,Mass
29054,00007569_019.png,18,7569,53,0,0,2992,2991,0.143000,0.143000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
109014,00029599_001.png,1,29599,57,0,0,2021,2021,0.194311,0.194311,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
106719,00028785_002.png,2,28785,20,0,1,3056,2544,0.139000,0.139000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
80259,00019726_007.png,7,19726,66,0,0,2992,2991,0.143000,0.143000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
45461,00011655_000.png,0,11655,42,1,0,2992,2991,0.143000,0.143000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
68105,00016805_007.png,7,16805,66,1,0,2294,2991,0.143000,0.143000,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
28486,00007431_001.png,1,7431,48,1,1,2500,2048,0.171000,0.171000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
36156,00009548_000.png,0,9548,7,0,0,2048,2500,0.168000,0.168000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
70900,00017491_000.png,0,17491,22,0,0,2500,2048,0.168000,0.168000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
16446,00004381_013.png,40,4381,29,0,1,3056,2544,0.139000,0.139000,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


In [191]:
xray_test_other_view_df = pd.pivot_table(xray_test_df, index=['Patient ID', 'Follow-up #', 'Image Index'])
xray_test_other_view_df.head(20)

Atelectasis  Cardiomegaly  \
Patient ID Follow-up # Image Index                                   
1          0           00000001_000.png            0             1   
           1           00000001_001.png            0             1   
           2           00000001_002.png            0             1   
2          0           00000002_000.png            0             0   
3          0           00000003_001.png            0             0   
           1           00000003_002.png            0             0   
           2           00000003_003.png            0             0   
           3           00000003_004.png            0             0   
           4           00000003_005.png            0             0   
           5           00000003_006.png            0             0   
           6           00000003_007.png            0             0   
           7           00000003_000.png            0             0   
4          0           00000004_000.png            0             0   
5          0           00000005_000.png            0             0   
           1           00000005_001.png            0             0   
           2           00000005_002.png            0             0   
           3           00000005_003.png            0             0   
           4           00000005_004.png            0             0   
           5           00000005_005.png            0             0   
           6           00000005_006.png            0             0   

                                         Consolidation  Edema  Effusion  \
Patient ID Follow-up # Image Index                                        
1          0           00000001_000.png              0      0         0   
           1           00000001_001.png              0      0         0   
           2           00000001_002.png              0      0         1   
2          0           00000002_000.png              0      0         0   
3          0           00000003_001.png              0      0         0   
           1           00000003_002.png              0      0         0   
           2           00000003_003.png              0      0         0   
           3           00000003_004.png              0      0         0   
           4           00000003_005.png              0      0         0   
           5           00000003_006.png              0      0         0   
           6           00000003_007.png              0      0         0   
           7           00000003_000.png              0      0         0   
4          0           00000004_000.png              0      0         0   
5          0           00000005_000.png              0      0         0   
           1           00000005_001.png              0      0         0   
           2           00000005_002.png              0      0         0   
           3           00000005_003.png              0      0         0   
           4           00000005_004.png              0      0         0   
           5           00000005_005.png              0      0         0   
           6           00000005_006.png              0      0         0   

                                         Emphysema  Fibrosis  Height]  Hernia  \
Patient ID Follow-up # Image Index                                              
1          0           00000001_000.png          0         0     2749       0   
           1           00000001_001.png          1         0     2729       0   
           2           00000001_002.png          0         0     2048       0   
2          0           00000002_000.png          0         0     2048       0   
3          0           00000003_001.png          0         0     2048       1   
           1           00000003_002.png          0         0     2500       1   
           2           00000003_003.png          0         0     2991       1   
           3           00000003_004.png          0         0     2048       1   
           4           00000003_005.png

In [195]:
xray_list_df.isna().count()

Image Index      984
Finding Label    984
Bbox [x          984
y                984
w                984
h]               984
Unnamed: 6       984
Unnamed: 7       984
Unnamed: 8       984
dtype: int64

In [197]:
xray_list_df = xray_list_df.dropna(axis=1)
xray_list_df.sample(10)

,Image Index,Finding Label,Bbox [x,y,w,h]
303,00008399_007.png,Cardiomegaly,363.005291,379.259259,547.216931,371.674074
127,00023116_005.png,Atelectasis,641.489947,438.857143,179.877249,158.205291
196,00017511_006.png,Cardiomegaly,338.983051,423.864407,397.016949,334.101695
733,00012415_002.png,Nodule,398.764021,534.213757,74.768254,86.687831
690,00018984_000.png,Nodule,825.701587,309.908995,72.601058,115.944974
801,00014125_042.png,Pneumonia,253.155556,293.869045,556.373333,434.631111
699,00004911_018.png,Nodule,695.669841,319.661376,100.774603,88.855026
175,00007037_000.png,Cardiomegaly,289.084746,487.358200,500.067797,385.084746
782,00022084_000.png,Pneumonia,209.920000,347.344601,763.448889,468.764444
523,00000211_016.png,Infiltrate,237.307937,465.947090,230.806349,328.330159


In [198]:
def search_me2(df_series, term):
    return 1 if term in df_series else 0

In [200]:
for each_disease in my_unique_list:
    xray_list_df[each_disease] = xray_list_df["Finding Label"].apply(search_me2, args=(each_disease,))
xray_list_df = xray_list_df.drop('Finding Label', axis=1)

In [202]:
xray_list_df.sample(20)

,Image Index,Bbox [x,y,w,h],Consolidation,Pleural_Thickening,Infiltration,Cardiomegaly,Pneumonia,Hernia,Edema,Fibrosis,Atelectasis,No Finding,Nodule,Effusion,Emphysema,Pneumothorax,Mass
156,00018419_001.png,746.598942,642.573545,115.944974,52.012698,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
632,00019499_000.png,723.843386,312.076190,111.610582,97.523810,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
178,00013249_031.png,383.457627,478.680234,403.525424,258.169492,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
662,00013991_000.png,381.426455,370.590476,106.192593,95.356614,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
727,00017028_000.png,145.202116,579.724868,76.935450,58.514286,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
712,00017098_003.png,300.156614,284.986243,58.514286,76.935450,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
496,00029039_020.png,162.133333,117.513490,670.151111,555.235556,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
491,00019625_002.png,190.577778,312.073490,656.497778,409.600000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
792,00017952_008.png,198.542222,215.362378,220.728889,440.320000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
403,00012973_005.png,122.446561,62.848677,279.568254,762.852910,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
